<a href="https://colab.research.google.com/github/chokima/machine-learning/blob/main/Weights_and_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [2]:
import wandb
import random

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()


wandb: Currently logged in as: muchokimark17 (muchokimark17-student). Use `wandb login --relogin` to force relogin


acc,▁▅▅▆▆▇▇█
loss,▇█▄▃▃▁▃▁
acc,0.83031
loss,0.20389


In [5]:
import random

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="basic-intro",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })

  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset

      # 🐝 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})

  # Mark the run as finished
  wandb.finish()

acc,▁▂▅▅█▇▇█
loss,█▆▁▁▁▂▁▁
acc,0.8366
loss,0.2333


acc,▁▆▇▇▇██▇
loss,█▄▃▄▃▁▂▁
acc,0.80124
loss,0.14753


acc,▁▆█▇▇▇█▇
loss,█▄▄▁▁▁▁▂
acc,0.7026
loss,0.27917


acc,▁▅▇▆████
loss,█▅▁▁▂▂▂▁
acc,0.73162
loss,0.25866


acc,▁▅▃▇█▆▇▆
loss,█▅▂▂▂▁▂▁
acc,0.8541
loss,0.04732


In [6]:
#@title
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset,
                                         batch_size=batch_size,
                                         shuffle=True if is_train else False,
                                         pin_memory=True, num_workers=2)
    return loader

def get_model(dropout):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

In [7]:
# Launch 5 experiments, trying different dropout rates
for _ in range(5):
    # 🐝 initialise a wandb run
    wandb.init(
        project="pytorch-intro",
        config={
            "epochs": 10,
            "batch_size": 128,
            "lr": 1e-3,
            "dropout": random.uniform(0.01, 0.80),
            })

    # Copy your config
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {"train/train_loss": train_loss,
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                       "train/example_ct": example_ct}

            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb
                wandb.log(metrics)

            step_ct += 1

        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss,
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})

        print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # 🐝 Close your wandb run
    wandb.finish()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 9025911.08it/s] 


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1259560.31it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▆▃▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▄▅▆▆▇▇▇██
val/val_loss,█▅▄▃▃▂▂▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.17173
val/val_accuracy,0.9465
val/val_loss,0.16722


Train Loss: 0.433, Valid Loss: 0.283599, Accuracy: 0.92
Train Loss: 0.171, Valid Loss: 0.231214, Accuracy: 0.93
Train Loss: 0.121, Valid Loss: 0.199388, Accuracy: 0.94
Train Loss: 0.136, Valid Loss: 0.189517, Accuracy: 0.94
Train Loss: 0.074, Valid Loss: 0.170017, Accuracy: 0.94
Train Loss: 0.105, Valid Loss: 0.165055, Accuracy: 0.95
Train Loss: 0.200, Valid Loss: 0.158087, Accuracy: 0.95
Train Loss: 0.070, Valid Loss: 0.158147, Accuracy: 0.95
Train Loss: 0.061, Valid Loss: 0.148396, Accuracy: 0.95
Train Loss: 0.016, Valid Loss: 0.146216, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▄▃▄▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▃▅▅▆▇████
val/val_loss,█▅▄▃▂▂▂▂▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.01595
val/val_accuracy,0.951
val/val_loss,0.14622


Train Loss: 0.607, Valid Loss: 0.347136, Accuracy: 0.90
Train Loss: 0.312, Valid Loss: 0.277134, Accuracy: 0.92
Train Loss: 0.289, Valid Loss: 0.247806, Accuracy: 0.92
Train Loss: 0.301, Valid Loss: 0.236012, Accuracy: 0.93
Train Loss: 0.278, Valid Loss: 0.221503, Accuracy: 0.93
Train Loss: 0.283, Valid Loss: 0.210943, Accuracy: 0.93
Train Loss: 0.305, Valid Loss: 0.206025, Accuracy: 0.94
Train Loss: 0.302, Valid Loss: 0.195520, Accuracy: 0.94
Train Loss: 0.206, Valid Loss: 0.194638, Accuracy: 0.94
Train Loss: 0.145, Valid Loss: 0.187877, Accuracy: 0.94


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▆▄▄▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▃▁▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▃▁▂
val/val_accuracy,▁▄▅▅▆▇▇▇██
val/val_loss,█▅▄▃▂▂▂▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.14456
val/val_accuracy,0.9405
val/val_loss,0.18788


In [8]:
# Start a wandb run
wandb.init(project="pytorch-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f'Accuracy is: {accuracy}, {acc_threshold}')

    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title='Low Accuracy',
            text=f'Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}',
        )
        print('Alert triggered')
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 0.819, 0.3
Accuracy is: 1.145, 0.3
Accuracy is: 1.025, 0.3
Accuracy is: 0.724, 0.3
Accuracy is: 0.759, 0.3
Accuracy is: 0.848, 0.3
Accuracy is: 0.719, 0.3
Accuracy is: 1.246, 0.3
Accuracy is: 0.939, 0.3
Accuracy is: 0.877, 0.3
Accuracy is: 0.801, 0.3
Accuracy is: 1.686, 0.3
Accuracy is: 1.024, 0.3
Accuracy is: 1.228, 0.3
Accuracy is: 1.011, 0.3
Accuracy is: 1.092, 0.3
Accuracy is: 1.611, 0.3
Accuracy is: 1.19, 0.3
Accuracy is: 1.526, 0.3
Accuracy is: 0.539, 0.3
Accuracy is: 1.476, 0.3
Accuracy is: 0.803, 0.3
Accuracy is: 1.38, 0.3
Accuracy is: 0.883, 0.3
Accuracy is: 0.476, 0.3
Accuracy is: 0.982, 0.3
Accuracy is: 1.05, 0.3
Accuracy is: 0.86, 0.3
Accuracy is: 1.008, 0.3
Accuracy is: 1.314, 0.3
Accuracy is: 0.459, 0.3
Accuracy is: 0.73, 0.3
Accuracy is: 0.876, 0.3
Accuracy is: 1.037, 0.3
Accuracy is: 1.552, 0.3
Accuracy is: 0.529, 0.3
Accuracy is: 0.66, 0.3
Accuracy is: 0.863, 0.3
Accuracy is: 1.311, 0.3
Accuracy is: 1.229, 0.3
Accuracy is: 1.654, 0.3
Accuracy is: 1.135, 0.

Accuracy,▃▅▄▃▃▃▅▄▃▇▅▄▄▅▆▂▆▆▄▂▄▃▄▂▃▄▆▂▃▃▅▇▅▄▄▃▄█▁▁
Accuracy,0.3
